In [2]:
### Code written using tensorflow

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the pixel values to the range of 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the input images to include a single channel (grayscale)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the CNN model
model = Sequential()

# Add a convolutional layer with 16 filters, each of size 3x3
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))

# Add a max pooling layer with pool size 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer with 32 filters, each of size 3x3
model.add(Conv2D(32, (3, 3), activation='relu'))

# Add a max pooling layer with pool size 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the previous layer
model.add(Flatten())

# Add a fully connected layer with 128 units
model.add(Dense(128, activation='relu'))

# Add the final output layer with 10 units (one for each class)
model.add(Dense(10, activation='softmax'))

# Print the model summary to see the number of trainable parameters
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=128)

# Evaluate the model on the test set
_, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy:", test_accuracy)


11490434/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                         

In [3]:
### code written using pytorch

In [4]:
  import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Load the MNIST dataset
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define the CNN model
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 32, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(800, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax(dim=1)
)

# Print the model summary to see the number of trainable parameters
print(model)

# Define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
min_val_accuracy = 0.994  # Minimum validation accuracy required
best_model = None

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        output = model(images)  # Forward pass
        loss = criterion(output, labels)  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights
    
    # Evaluate the model on the validation set
    model.eval()  # Set the model to evaluation mode
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_accuracy = val_correct / val_total
    print(f"Epoch [{epoch+1}/{num_epochs}] | Validation Accuracy: {val_accuracy:.4f}")
    
    # Check if the current model has the minimum required validation accuracy
    if val_accuracy >= min_val_accuracy:
        best_model = model
        break

# Use the best model for testing
if best_model is not None:
    model = best_model

# Evaluate the model on the test set
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = test_correct / test_total
print("Test Accuracy:", test_accuracy)


100%|██████████| 9912422/9912422 [00:00<00:00, 109828850.20it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 25757111.17it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25375506.88it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18126097.78it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Sequential(
  (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=800, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=10, bias=True)
  (10): LogSoftmax(dim=1)
)
Epoch [1/10] | Validation Accuracy: 0.9737
Epoch [2/10] | Validation Accuracy: 0.9824
Epoch [3/10] | Validation Accuracy: 0.9839
Epoch [4/10] | Validation Accuracy: 0.9869
Epoch [5/10] | Validation Accuracy: 0.9875
Epoch [6/10] | Validation Accuracy: 0.9878
Epoch [7/10] | Validation Accuracy: 0.9892
Epoch [8/10] | Validation Accuracy: 0.9895
Epoch [9/10] | Validation Accuracy: 0.9908
Epoch [10/10] | Vali